In [1]:
import pandas as pd
import os
import re

In [ ]:
df = pd.read_csv(r'\meds_pipeline\data\1_trusted\database_brasindice.csv')

In [4]:
# Função para extrair a primeira palavra
def extrair_primeira_palavra(descricao):
    if pd.isna(descricao):  # Verificar valores nulos
        return descricao
    palavras = descricao.split()  # Dividir a descrição em palavras
    return palavras[0]  # Retornar apenas a primeira palavra

In [5]:
df.columns

Index(['codigo_tiss', 'codigo_tuss', 'produto', 'laboratorio', 'valor',
       'cobranca'],
      dtype='object')

In [6]:
df['produto_limpo'] = df['produto'].apply(extrair_primeira_palavra)

In [8]:
print(df[['produto', 'produto_limpo']].sample(10))


                                                 produto produto_limpo
15373  VALSARTANA + HIDROCLOROTIAZIDA - GENERICO 80 +...    VALSARTANA
4244   CLORIDRATO DE METFORMINA - GENERICO (RESTRITO ...    CLORIDRATO
10609              MUCOSOLVAN GTS. 1 X 50 ML. BOEHRINGER    MUCOSOLVAN
12738          PROAZITAX 60MG SOL.INJ.FA 1,5ML+DIL.4,5ML     PROAZITAX
1895                      BALSAMO BENGUE POM. BISN. 60 G       BALSAMO
2191                        BIMAGAN 0,3 MG/ML. FR. 3 ML.       BIMAGAN
749    ALLEGRA 180 MG. CT. BL. AL. PLAS. TRANS. X 30 ...       ALLEGRA
12785                    PROLEPTOL 150MG 30CAPS GEL.DURA     PROLEPTOL
1325   ARIPIPRAZOL - GENERICO 10 MG. CT. BL. AL./AL. ...   ARIPIPRAZOL
828     ALPRAZOLAM - GENERICO 0,5 MG. BL. 30 CPRS. TEUTO    ALPRAZOLAM


In [9]:
# Função para extrair a mg ou ml 
def extrair_mg_total(descricao):
   descricao = str(descricao).lower()
   # Padronizações
   descricao = descricao.replace("miligrama", "mg").replace("miligramas", "mg").replace('mcg', 'mg')
   descricao = descricao.replace("mililitro", "ml").replace("mililitros", "ml")
   descricao = descricao.replace("grama", "g").replace("gramas", "g")

   # Corrigir decimais quebrados: "38 5 mg" → "38.5 mg", "2 5 ml" → "2.5 ml"
   # Essa regex trata qualquer número + espaço + número curto + mg/ml/g
   descricao = re.sub(r'(\d+)\s+(\d{1,2})\s*(mg|ml|g)', r'\1.\2 \3', descricao)
   # Corrigir milhar quebrado: "1 000 mg" → "1000 mg" (somente se for 3 dígitos após)
   descricao = re.sub(r'(?<!\d)(\d{1,2})\s+(\d{3})\s*mg', r'\1\2 mg', descricao)
   # Casos mg/ml: "X mg/ml Y ml" → calcular mg total
   padrao_mg_ml = re.search(r'(\d+(?:[\.,]\d+)?)\s*mg\s*(?:\/|\s*)ml.*?(\d+(?:[\.,]\d+)?)\s*ml', descricao)
   if padrao_mg_ml:
       mg_por_ml = float(padrao_mg_ml.group(1).replace(',', '.'))
       ml_total = float(padrao_mg_ml.group(2).replace(',', '.'))
       return round(mg_por_ml * ml_total, 2)
   # Casos com gramas → mg
   valores_g = re.findall(r'(\d+(?:[\.,]\d+)?)\s*g', descricao)
   if valores_g:
       valor_g = float(valores_g[-1].replace(',', '.'))
       return round(valor_g * 1000, 2)
   # Regra final: pegar o primeiro número antes de "mg"
   texto_antes_de_mg = descricao.split("mg")[0]
   numeros = re.findall(r'\d+(?:[\.,]\d+)?', texto_antes_de_mg)
   if numeros:
       return round(float(numeros[0].replace(',', '.')), 2)
   return None

In [10]:
df['mg_por_frasco'] = df['produto'].apply(extrair_mg_total)

In [12]:
print(df[['produto','mg_por_frasco']].sample(10))

                                                 produto  mg_por_frasco
10830           NEO ITRAX 100 MG CX. 1 BL. X 4 CAPS. NEO          100.0
11996  PANTOPRAZOL - GENERICO 40 MG. 6 BL. X 7 CPRS. ...           40.0
11327                         NUBAIN 10MG/ML 25 AMP 1 ML           10.0
1076   AMOXILINA TRI-HIDRATADA+CLAVULANATO POTASSIO 5...           50.0
11639  OPTIMARK (RESTRITO HOSP.) SERINGA PREENCH. 15 ...           15.0
12957    QUET 100 MG. BL. AL. PLAS. OPC. X 30 CPRS. REV.          100.0
1258       APROZIDE 150 MG. + 12,5 MG. BL. 30 CPRS. REV.          150.0
10777                 NAUSEDRON 8 MG. CX. 1 FR. 10 CPRS.            8.0
3798              CLOMETINE 8MG 10CAPS DURA LIB.PROLONG.            8.0
4324   CLORIDRATO DE NARATRIPTANA - GENERICO 2,5 MG. ...            2.5


In [13]:
# Nova coluna juntando o nome do produto com a quantidade mg ou ml
def juntar_colunas_prod_mg(row):
    descricao = row['produto_limpo']
    mg = row['mg_por_frasco']
 
    if pd.notna(mg):
         # Se for um número inteiro, mostra sem decimal
        if mg == int(mg):
            mg_formatado = int(mg)
        else:
            mg_formatado = round(mg, 2)  # ou 1 casa decimal se preferir
        return f"{descricao} {mg_formatado}mg"
    else:
        return descricao
    
df['produto_tratado'] = df.apply(juntar_colunas_prod_mg, axis=1)

In [14]:
print(df[['produto','produto_tratado']].sample(10))

                                                 produto    produto_tratado
8123   HEMIFUMARATO DE QUETIAPINA - GENERICO 25 MG. 2...  HEMIFUMARATO 25mg
5297                           DEPURA 1000UI 30CPDS REV.      DEPURA 1000mg
3606   CITRATO DE SILDENAFILA - GENERICO 50 MG. 2 CPR...       CITRATO 50mg
14215      TADALAFILA - GENERICO 20 MG. BL. 1 CPRS. REV.    TADALAFILA 20mg
1196                        ANGIPRESS CD 25+125MG 15CPDS     ANGIPRESS 25mg
4572   CLORIDRATO DE TRAMADOL - GENERICO 50 MG. CT. B...    CLORIDRATO 50mg
13199                 RILAN SOL. NASAL 4% FR. DOS. 13 ML          RILAN 4mg
5189   DDAVP (RESTRITO HOSP.) 4 MCG/ML CART. 10 AMPS....          DDAVP 4mg
728               ALIVIUM 400 MG. BL. 10 CPRS. MANTECORP      ALIVIUM 400mg
11909                 PALOCYT 0,05MG/ML SOL.INJ.20FA 5ML     PALOCYT 0.25mg


In [15]:
colunas = ['codigo_tuss', 'produto', 'produto_tratado','laboratorio', 'valor']
df = df[colunas]

In [16]:
df.to_csv(r'C:\Users\Thyana De Lara\Documents\TCC\meds_pipeline\data\2_refined\medication_data_final.csv',index=False)